# 6장 모델 평가와 하이퍼파라미터 튜닝의 모범 사례

In [1]:
from IPython.display import Image
%matplotlib inline

## 파이프라인을 사용한 효율적인 워크플로

새로운 데이터의 스케일을 조정하고 압축하기 위해 학습한 파라미터 재사용 해야함

사이킷 런의 Pipeline 클래스를 사용하면 여러 개의 변환 단계를 포함한 모델을 학습하고 새로운 데이터에 대한 예측을 만들 수 있음

### 위스콘신 유방암 데이터셋

In [2]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/'
                 'machine-learning-databases'
                 '/breast-cancer-wisconsin/wdbc.data', header=None)

# UCI 머신 러닝 저장소에서 유방암 데이터셋을 다운로드할 수 없을 때
# 다음 주석을 해제하고 로컬 경로에서 데이터셋을 적재하세요:

# df = pd.read_csv('wdbc.data', header=None)

#32개의 컬럼...
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
# 원본 문자열표현 정수 변환
from sklearn.preprocessing import LabelEncoder

X = df.loc[:, 2:].values
y = df.loc[:, 1].values
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_

array(['B', 'M'], dtype=object)

In [4]:
# 매핑확인
le.transform(['M', 'B'])

array([1, 0])

In [6]:
# 훈련(80%), 테스트(20%) 데이터셋 나눔
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y,
                     test_size=0.20,
                     stratify=y,
                     random_state=1)

### 파이프라인으로 변환기와 추정기 연결

In [7]:
'''
주성분 분석(PCA)으로 30여개의 차원을 2차원 부분 공간으로 데이터 압축
훈련 데이터셋과 테스트 데이터셋을 각각 학습하고 변환하는 단계를 구성하는 대신 StandardScaler, PCA, LogisticRegression 객체를 하나의 파이프라인으로 연결

Pipeline 객체의 fit 메서드를 호출하면 데이터가 중간 단계에 있는 모든 변환기의 fit 메서드와 transform 메서드를 차례로 거쳐 추정기 객체(파이프라인의 마지막 단계)에 도달. 추정기는 변환된 훈련 데이터셋을 사용하여 학습
1. pipe_lr 파이프라인의 fit 메서드를 호출할 때 먼저 훈련 데이터셋에 StandardScaler 의 fit 메서드와 transform 메서드가 호출(StandardScaler())
2. 변환된 훈련 데이터는 파이프라인의 다음 요소인 PCA 객체로 전달, 이전 단계와 비슷하게 스케일 조정된 입력 데이터에 PCA의 fit 메서드와 transform 메서드가 호출 (PCA(n_components=2))
3. 파이프라인의 최종 요소인 추정기에 훈련 데이터가 전달(LogisticRegression)
'''
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression())

pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
test_acc = pipe_lr.score(X_test, y_test)
print(f'테스트 정확도: {test_acc:.3f}')

테스트 정확도: 0.956


In [9]:
pipe_lr

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=2)),
                ('logisticregression', LogisticRegression())])

In [8]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch06/figures/06_01.png', width=500)

## k-폴드 교차검증을 사용한 모델 성능평가(k-겹 교차검증)

보편적인 교차 검증 기법인 홀드아웃 방법(holdout method)과 k-겹 교차 검증(k-fold cross-validation)을 학습

### 홀드아웃 방법

In [10]:
'''
홀드 아웃 방법은 초기 데이터셋을 별도의 훈련 데이터셋과 테스트 데이터셋으로 나눔
일반적인 머신 러닝 애플리케이션에서는 처음 본 데이터에서 예측 성능을 높이기 위해 하이퍼파라미터를 튜닝하고 비교 하는 것을 '모델 선택'
모델 선택은 주어진 분류 문제에서 튜닝할 파라미터(또는 하이퍼파라미터)의 최적 값을 선택해야 하는 것을 의미
모델 선택에 같은 테스트 데이터셋을 반복해서 재사용하면 훈련 데이터셋의 일부가 되는 셈이고 결국 모델은 과대적합 됨
모델 선택에 홀드아웃 방법을 사용하는 가장 좋은 방법은 데이터를 훈련 데이터셋, 검증 데이터셋, 테스트 데이터셋 세 개의 부분으로 나눔
훈련 데이터셋은 여러 가지 모델을 훈련하는 데 사용. 검증 데이터셋에 대한 성능은 모델 선택에 사용. 테스트 데이터셋을 분리했기 때문에 새로운 데이터에 대한 일반화 능력을 덜 편향되게 추정할 수 있는 장점
만족할 만한 하이퍼파라미터 값을 얻었다면 테스트 데이터셋에서 모델의 일반화 성능을 추정
'''
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch06/figures/06_02.png', width=500)

### k폴드 교차 검증

In [11]:
'''
k-폴드 교차 검증에서는 중복을 허용하지 않고 훈련 데이터셋을 k개의 폴드(fold)로 랜덤하게 나눔.
k-1개의 폴드(이를 훈련 폴드(training fold)라고 함)로 모델을 훈련하고 나머지 하나의 폴드(이를 테스트 폴드(test fold)라고 함)로 성능을 평가
이 과정을 k번 반복하여 k개의 모델과 성능 추정을 얻음
서로 다른 독립적인 폴드에서 얻은 성능 추정을 기반으로 모델의 평균 성능을 계산. 일반적으로 모델 튜닝에 k-겹 교차 검증을 사용
즉, 테스트 데이터셋에서 모델의 성능을 평가할 때 만족할 만한 일반화 성능을 내는 최적의 하이퍼파라미터 값을 찾기 위해 사용
하이퍼파라미터를 찾은 후, 전체 훈련 데이터셋을 사용하여 모델을 다시 훈련. 독립적인 테스트 데이터셋을 사용하여 최종 성능 추정
k폴드 교차검증 후 전체 훈련 데이터셋으로 학습하는 이유는 하나의 최종 모델이 필요하고, 학습 알고리즘이 더 정확하고 안정적인 모델을 만듦
k폴드 교차 검증에서 모든 데이터 포인트가 평가에 사용되기 때문에 검증 세트를 사용하는 홀드아웃 방법보다 k폴드 교차 검증이 데이터셋을 더 잘 활용
'''
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch06/figures/06_03.png', width=500)

In [12]:
'''
k폴드 교차 검증에서 좋은 기본값은 k = 10
론 코하비(Ron Kohavi)는 여러 종류의 실제 데이터셋에서 수행한 실험을 통해 10-겹 교차 검증이 가장 뛰어난 편향 -분산 트레이드오프를 가진다고 제안
작은 훈련 데이터셋에는 k값을 늘려서 훈련 데이터가 각 반복에 사용되어 일반화 성능을 추정할 때 낮은 편향을 만듦
큰 훈련 데이터셋에는 k값을 줄여서 모델의 평균 성능을 정확하게 추정하고, 폴드마다 학습하고 평가하는 계산 비용을 줄임
'''
import numpy as np
from sklearn.model_selection import StratifiedKFold


kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)

scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)

    print(f'폴드: {k+1:02d}, '
          f'클래스 분포: {np.bincount(y_train[train])}, '
          f'정확도: {score:.3f}')

mean_acc = np.mean(scores)
std_acc = np.std(scores)
print(f'\nCV 정확도: {mean_acc:.3f} +/- {std_acc:.3f}')

폴드: 01, 클래스 분포: [256 153], 정확도: 0.935
폴드: 02, 클래스 분포: [256 153], 정확도: 0.935
폴드: 03, 클래스 분포: [256 153], 정확도: 0.957
폴드: 04, 클래스 분포: [256 153], 정확도: 0.957
폴드: 05, 클래스 분포: [256 153], 정확도: 0.935
폴드: 06, 클래스 분포: [257 153], 정확도: 0.956
폴드: 07, 클래스 분포: [257 153], 정확도: 0.978
폴드: 08, 클래스 분포: [257 153], 정확도: 0.933
폴드: 09, 클래스 분포: [257 153], 정확도: 0.956
폴드: 10, 클래스 분포: [257 153], 정확도: 0.956

CV 정확도: 0.950 +/- 0.014
